In [1]:
import math
import requests
import pandas as pd
from ngboost import NGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import json
from ngboost.ngboost import NGBoost
from ngboost.scores import CRPS, MLE
from ngboost.distns import Normal, LogNormal
import numpy as np

def smape_score(y_true, y_pred):
    return (100/len(y_true)) * sum(2 * abs(y_true - y_pred) / (abs(y_true) + abs(y_pred)))

c:\Users\itopya\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\itopya\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\itopya\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Nominatim, open-source geocoding service
import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_app")

def haversine(lat1, lon1, lat2, lon2): # Distance calculation
    r = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = r * c
    return distance

def missing_mahalle_filling_geo(df_missing, df_train_coordinates):
    
    categoric = ["ILI", "ILCESI", "MAHALLESI", "ADRES"]
    df_missing[categoric] = df_missing[categoric].astype('str')
    for mahalle in df_missing["MAHALLESI"].unique():
        
        if mahalle not in transform_dict.get("MAHALLESI", {}).keys():
            
            il = df_missing.loc[df_missing["MAHALLESI"] == mahalle, ["ILI"]]["ILI"].iloc[0]
            ilce = df_missing.loc[df_missing["MAHALLESI"] == mahalle, ["ILCESI"]]["ILCESI"].iloc[0]
            yakin_mahalleler = pd.DataFrame(columns=["KAYIP_MAHALLE"])
            
            missing_adres = mahalle + " Mahallesi, " + ilce +", "+ il
            location = geolocator.geocode(missing_adres)
            missing_adress_lat, missing_adress_lng = location.latitude, location.longitude
            
            ayni_il_ilce = df_train_coordinates.loc[(df_train_coordinates["ILI"]==il) & (df_train_coordinates["ILCESI"]==ilce)]
            
            for yakin_mahalle in ayni_il_ilce["MAHALLESI"].unique():
                
                ayni_ilce_lat, ayni_ilce_lng = df_train_coordinates.loc[df_train_coordinates["MAHALLESI"] == yakin_mahalle, ["LATITUDE", "LONGITUDE"]].iloc[0]
                distance = haversine(missing_adress_lat, missing_adress_lng, ayni_ilce_lat, ayni_ilce_lng)
                yakin_mahalleler = pd.concat([yakin_mahalleler, pd.DataFrame({"KAYIP_MAHALLE": [mahalle], "AYNI_ILCE_MAHALLE": [yakin_mahalle], "DISTANCE": [distance]})], ignore_index=True)

            yakin_mahalleler["DISTANCE"] = pd.to_numeric(yakin_mahalleler["DISTANCE"], errors='coerce')
            print(yakin_mahalleler.sort_values(by='DISTANCE', ascending=True))
            
            min_distance_mahalle = yakin_mahalleler.loc[yakin_mahalleler["DISTANCE"] > 0, "DISTANCE"].min()
            closest_mahalle = yakin_mahalleler.loc[yakin_mahalleler["DISTANCE"] == min_distance_mahalle, "AYNI_ILCE_MAHALLE"].iloc[0]
            code_of_missing_mahalle = transform_dict['MAHALLESI'][closest_mahalle]
            
            df_missing.loc[df_missing["MAHALLESI"]==mahalle, ["MAHALLESI", "ADRES"]] = code_of_missing_mahalle
            print(f"Code of missing mahalle is: ", code_of_missing_mahalle)
    
    return df_missing

In [3]:
df = pd.read_csv("train.csv")

df["MAHALLESI"] = df["MAHALLESI"].str.replace("İ", "I")

In [4]:
categoric = ["ILI", "ILCESI", "MAHALLESI", "ADRES"]
df[categoric] = df[categoric].astype('str')

transform_dict = {}
for col in df[["ILI", "ILCESI", "MAHALLESI", "ADRES"]]:
    cats = pd.Categorical(df[col]).categories
    d = {}
    for i, cat in enumerate(cats):
        d[cat] = i
    transform_dict[col] = d
    
inverse_transform_dict = {}
for col, d in transform_dict.items():
    inverse_transform_dict[col] = {int(v):k for k, v in d.items()}

with open('encode_transform_genel.json', 'w') as f:
    json.dump(transform_dict, f)
with open('inverse_encode_transform_genel.json', 'w') as f:
    json.dump(inverse_transform_dict, f)

with open('encode_transform_genel.json') as f:
    encode_transform = json.load(f)

with open('inverse_encode_transform_genel.json') as f:
    inverse_encode_transform = json.load(f)

for col in inverse_encode_transform:
    inverse_encode_transform[col] = {int(k): v for k, v in inverse_encode_transform[col].items()}

df = df.replace(encode_transform)

In [5]:
df.MAHALLESI.unique()

array(['KAVAKPINAR', 'KURTKÖY', 'VELIBABA', 'KAYNARCA', 'BAHÇELIEVLER',
       'FEVZI ÇAKMAK', 'GÜZELYALI', 'YENIŞEHIR', 'ÇINARDERE',
       'GÜLLÜ BAĞLAR', 'YAYALAR', 'DOĞU', 'ÇAMÇEŞME', 'AHMET YESEVI',
       'ÇAMLIK', 'BATI', 'HARMANDERE', 'SÜLÜNTEPE', 'ESENLER',
       'ORHANGAZI', 'DUMLUPINAR', 'SAPAN BAĞLARI', 'DOLAYOBA'],
      dtype=object)

In [6]:
df_uniques = df[['ILI', 'ILCESI', 'MAHALLESI']].drop_duplicates().reset_index(drop=True)

In [7]:
df_coordinates_nominatim = pd.DataFrame()
for index, row in df_uniques.iterrows():
    location = geolocator.geocode(row['MAHALLESI'] + ' Mahallesi, ' + row['ILCESI'] + ', ' + row['ILI'])
    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
        address = location.address
        
        # yeni bir satır oluştur ve verileri ekle
        new_row = pd.DataFrame({'ILI': [row['ILI']],
                        'ILCESI': [row['ILCESI']],
                        'MAHALLESI': [row['MAHALLESI']],
                        'LATITUDE': [latitude],
                        'LONGITUDE': [longitude]})
        
        # dataframe'e ekle
        df_coordinates_nominatim = pd.concat([df_coordinates_nominatim, new_row], ignore_index=True)
    else:
        print(row['MAHALLESI'] + ' Mahallesi, ' + row['ILCESI'] + ', ' + row['ILI'] + ", adresi bulunamamıştır.")

With this lines of codes we find coordinates of the neighbourhoods of disctricts in the cities.

At this point we actually require another dataset for pulling coordinates of the neighbourhoods center points. Because in this dataset we actually may not have all of the actual neighbourhoods and may need all of them in order to work properly and efficently. 

For readers information: The Nominatim works poorly in spesific locations, yandex geo locator does a greater job but it's paid, therefore I had to use Nominatim free API.

In [8]:
df_coordinates_nominatim

,ILI,ILCESI,MAHALLESI,LATITUDE,LONGITUDE
0,İSTANBUL,PENDİK,KAVAKPINAR,40.881138,29.286823
1,İSTANBUL,PENDİK,KURTKÖY,40.914205,29.300273
2,İSTANBUL,PENDİK,VELIBABA,40.904488,29.254384
3,İSTANBUL,PENDİK,KAYNARCA,40.875242,29.255296
4,İSTANBUL,PENDİK,BAHÇELIEVLER,40.883911,29.235379
5,İSTANBUL,PENDİK,FEVZI ÇAKMAK,40.878071,29.263908
6,İSTANBUL,PENDİK,GÜZELYALI,40.860571,29.282181
7,İSTANBUL,PENDİK,YENIŞEHIR,40.931292,29.304338
8,İSTANBUL,PENDİK,ÇINARDERE,40.894557,29.239569
9,İSTANBUL,PENDİK,GÜLLÜ BAĞLAR,40.896909,29.274365


In [9]:
df_coordinates_nominatim.to_csv('pendik_koordinat.csv', index=False)

We have found other sources of information about our dataset and we need to optimize them for our dataset aswell.

In [10]:
features = pd.read_csv("features copy.csv")

We need only the existing values, we do not need empty rows.

In [11]:
features_filtered = features[~(features["total_area"] == 0) | (features["total_number"])]

In [12]:
features_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 192 entries, 1 to 644
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   area_id           192 non-null    int64  
 1   address           192 non-null    object 
 2   measurement_type  192 non-null    object 
 3   target_type       192 non-null    object 
 4   target            192 non-null    object 
 5   number            192 non-null    int64  
 6   total_number      192 non-null    int64  
 7   total_area        192 non-null    float64
dtypes: float64(1), int64(3), object(4)
memory usage: 13.5+ KB


In [13]:
features_filtered.address.unique()

array(['Kavakpınar Mahallesi, Pendik, İstanbul, Marmara Bölgesi, 34889, Türkiye',
       'Kurtköy Mahallesi, Pendik, İstanbul, Marmara Bölgesi, 34606, Türkiye',
       'Velibaba Mahallesi, Pendik, İstanbul, Marmara Bölgesi, Türkiye',
       'Kaynarca Mahallesi, Pendik, İstanbul, Marmara Bölgesi, Türkiye',
       'Bahçelievler Mahallesi, Pendik, İstanbul, Marmara Bölgesi, 34893, Türkiye',
       'Fevzi Çakmak Mahallesi, Pendik, İstanbul, Marmara Bölgesi, Türkiye',
       'Güzelyalı Mahallesi, Pendik, İstanbul, Marmara Bölgesi, Türkiye',
       'Yenişehir Mahallesi, Pendik, İstanbul, Marmara Bölgesi, Türkiye',
       'Çınardere Mahallesi, Pendik, İstanbul, Marmara Bölgesi, Türkiye',
       'Güllü Bağlar Mahallesi, Pendik, İstanbul, Marmara Bölgesi, Türkiye',
       'Yayalar Mahallesi, Pendik, İstanbul, Marmara Bölgesi, Türkiye',
       'Doğu Mahallesi, Pendik, İstanbul, Marmara Bölgesi, 34890, Türkiye',
       'Çamçeşme Mahallesi, Pendik, İstanbul, Marmara Bölgesi, Türkiye',
       'Ahme

We only need the neighbourhood names, so we need to filter them.

In [14]:
features_filtered[["number", "total_number"]].corr()

,number,total_number
number,1.0,1.0
total_number,1.0,1.0


This 2 variables are equals and has the same value for each row, because of that we need to remove one of them from our dataset.

In [15]:
features_filtered['address'] = features_filtered['address'].str.split(' Mahallesi').str[0]
features_filtered.drop(columns=["area_id", "total_number"], inplace=True)

C:\Users\itopya\AppData\Local\Temp\ipykernel_26716\3595669837.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_filtered['address'] = features_filtered['address'].str.split(' Mahallesi').str[0]
C:\Users\itopya\AppData\Local\Temp\ipykernel_26716\3595669837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_filtered.drop(columns=["area_id", "total_number"], inplace=True)


In [16]:
feature_df = features_filtered.copy()

In [17]:
feature_df.head(3)

,address,measurement_type,target_type,target,number,total_area
1,Kavakpınar,number,amenity,school,8,0.0
3,Kavakpınar,number,amenity,clinic,2,0.0
5,Kavakpınar,number,amenity,pharmacy,5,0.0


In [18]:
df_areas = feature_df[(feature_df["measurement_type"] == "area")].drop(["number", "measurement_type", "target_type"], axis=1,)
df_numbers = feature_df[(feature_df["measurement_type"] == "number")].drop(["total_area", "measurement_type", "target_type"], axis=1,)

df_numbers = df_numbers.pivot(index='address', columns='target', values='number').fillna(0).reset_index()
df_areas = df_areas.pivot(index='address', columns='target', values='total_area').fillna(0).reset_index()

In [19]:
df_numbers.columns

Index(['address', 'cinema', 'clinic', 'dentist', 'ferry_terminal',
       'fire_station', 'library', 'pharmacy', 'playground', 'police', 'school',
       'stadium', 'station'],
      dtype='object', name='target')

In [20]:
df_areas.columns

Index(['address', 'forest', 'hospital', 'marina', 'park', 'parking',
       'residential', 'theme_park'],
      dtype='object', name='target')

We check if the values are duplicated.

In [21]:
df_numbers.head(3)

target,address,cinema,clinic,dentist,ferry_terminal,fire_station,library,pharmacy,playground,police,school,stadium,station
0,Ahmet Yesevi,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0
1,Bahçelievler,0.0,1.0,1.0,0.0,1.0,1.0,7.0,0.0,0.0,9.0,0.0,0.0
2,Batı,2.0,2.0,2.0,1.0,0.0,3.0,16.0,2.0,0.0,2.0,0.0,1.0


In [22]:
df_areas.head(3)

target,address,forest,hospital,marina,park,parking,residential,theme_park
0,Ahmet Yesevi,8367.295569,0.000000,0.000000,55558.178555,2418.487567,1.827183e+06,0.0
1,Bahçelievler,0.000000,0.000000,0.000000,1997.733537,6571.636663,9.167007e+05,0.0
2,Batı,0.000000,3197.782197,268364.645012,328299.297397,72009.687664,6.684872e+05,0.0


I'm dropping theme_park and marina variables because it doesn't have meaningful info.

In [23]:
df_areas.drop(["theme_park","marina"], axis=1, inplace=True)

In [24]:
features_merged = pd.merge(df_numbers, df_areas, on="address")

In [25]:
features_merged.to_csv("features.csv", index=False)

In [26]:
features_merged = pd.read_csv("features.csv")

In [27]:
pd.options.display.float_format = '{:.2f}'.format
features_merged.head(3)

,address,cinema,clinic,dentist,ferry_terminal,fire_station,library,pharmacy,playground,police,school,stadium,station,forest,hospital,park,parking,residential
0,Ahmet Yesevi,0.00,1.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,3.00,0.00,0.00,8367.30,0.00,55558.18,2418.49,1827182.88
1,Bahçelievler,0.00,1.00,1.00,0.00,1.00,1.00,7.00,0.00,0.00,9.00,0.00,0.00,0.00,0.00,1997.73,6571.64,916700.74
2,Batı,2.00,2.00,2.00,1.00,0.00,3.00,16.00,2.00,0.00,2.00,0.00,1.00,0.00,3197.78,328299.30,72009.69,668487.22


In [28]:
features_merged.rename(columns = {"address": "MAHALLESI"}, inplace=True)
features_merged.columns # Changed the name of the address column so we can join them.

Index(['MAHALLESI', 'cinema', 'clinic', 'dentist', 'ferry_terminal',
       'fire_station', 'library', 'pharmacy', 'playground', 'police', 'school',
       'stadium', 'station', 'forest', 'hospital', 'park', 'parking',
       'residential'],
      dtype='object')

After creating our features dataset we now are going to merge these. But as we can see the values do not match just yet, we need to capitalize all of the characters in address column.

After that, we had to make them identical so they match with one another.

In [29]:
features_merged["MAHALLESI"] = features_merged["MAHALLESI"].apply(lambda x: x.upper())
df["MAHALLESI"] = df["MAHALLESI"].str.replace("İ", "I")

In [30]:
print(np.sort(df["MAHALLESI"].unique()))
print(np.sort(features_merged["MAHALLESI"].unique()))

['AHMET YESEVI' 'BAHÇELIEVLER' 'BATI' 'DOLAYOBA' 'DOĞU' 'DUMLUPINAR'
 'ESENLER' 'FEVZI ÇAKMAK' 'GÜLLÜ BAĞLAR' 'GÜZELYALI' 'HARMANDERE'
 'KAVAKPINAR' 'KAYNARCA' 'KURTKÖY' 'ORHANGAZI' 'SAPAN BAĞLARI' 'SÜLÜNTEPE'
 'VELIBABA' 'YAYALAR' 'YENIŞEHIR' 'ÇAMLIK' 'ÇAMÇEŞME' 'ÇINARDERE']
['AHMET YESEVI' 'BAHÇELIEVLER' 'BATI' 'DOĞU' 'DUMLUPINAR' 'ESENLER'
 'ESENYALI' 'FATIH' 'FEVZI ÇAKMAK' 'GÜLLÜ BAĞLAR' 'GÜZELYALI' 'HARMANDERE'
 'KAVAKPINAR' 'KAYNARCA' 'KURTKÖY' 'ORHANGAZI' 'SAPAN BAĞLARI' 'SÜLÜNTEPE'
 'VELIBABA' 'YAYALAR' 'YENI' 'YENIŞEHIR' 'ÇAMLIK' 'ÇAMÇEŞME' 'ÇINARDERE'
 'ŞEYHLI']


In [31]:
print(len(np.sort(df["MAHALLESI"].unique())))
print(len(np.sort(features_merged["MAHALLESI"].unique())))

23
26


In [32]:
df_coordinates_nominatim

,ILI,ILCESI,MAHALLESI,LATITUDE,LONGITUDE
0,İSTANBUL,PENDİK,KAVAKPINAR,40.88,29.29
1,İSTANBUL,PENDİK,KURTKÖY,40.91,29.30
2,İSTANBUL,PENDİK,VELIBABA,40.90,29.25
3,İSTANBUL,PENDİK,KAYNARCA,40.88,29.26
4,İSTANBUL,PENDİK,BAHÇELIEVLER,40.88,29.24
5,İSTANBUL,PENDİK,FEVZI ÇAKMAK,40.88,29.26
6,İSTANBUL,PENDİK,GÜZELYALI,40.86,29.28
7,İSTANBUL,PENDİK,YENIŞEHIR,40.93,29.30
8,İSTANBUL,PENDİK,ÇINARDERE,40.89,29.24
9,İSTANBUL,PENDİK,GÜLLÜ BAĞLAR,40.90,29.27


In [33]:
train_df = df.merge(features_merged, on="MAHALLESI")

Below, we have our submission dataset, this dataset contains different neighbourhoods from our actual "df" named dataset.

In [34]:
df_sub = pd.read_csv("sub.csv")
df_sub.loc[df_sub["MAHALLESI"] == "YENİALLE", "MAHALLESI"] = "YENİ MAHALLE"
df_sub["MAHALLESI"].unique()

df_sub["MAHALLESI"] = df_sub["MAHALLESI"].str.replace("İ", "I")
df_sub.loc[df_sub["MAHALLESI"] == "YENI MAHALLE", "MAHALLESI"] = "YENI"
# Manipulating the submission data for our main dataset.
df_sub.MAHALLESI.unique()

array(['YENI', 'FATIH', 'ESENYALI', 'ŞEYHLI'], dtype=object)

In [35]:
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ILI                 136 non-null    object 
 1   ILCESI              136 non-null    object 
 2   MAHALLESI           136 non-null    object 
 3   DAIRE_YASAL_M2      136 non-null    float64
 4   KAT_NO              136 non-null    int64  
 5   BINA_YASI           136 non-null    int64  
 6   KAT_ADEDI           136 non-null    int64  
 7   İNŞAAT_SEVİYESİ     136 non-null    int64  
 8   ASANSÖR             136 non-null    int64  
 9   DAIRE_SALON         136 non-null    int64  
 10  DAIRE_MUTFAK        136 non-null    float64
 11  DAIRE_ODA           136 non-null    int64  
 12  DAIRE_BANYO         136 non-null    int64  
 13  ADRES               136 non-null    object 
 14  DOLAR               136 non-null    float64
 15  TÜFE_KONUT          136 non-null    float64
 16  TÜFE_KİR

In [36]:
df_sub.head(3)

,ILI,ILCESI,MAHALLESI,DAIRE_YASAL_M2,KAT_NO,BINA_YASI,KAT_ADEDI,İNŞAAT_SEVİYESİ,ASANSÖR,DAIRE_SALON,...,TÜFE_KONUTLAİLGİLİ,TÜFE,ÜFE,KONUT_FAİZ,CAM,KİL,ÇİMENTO,METAL,KFE,KFB
0,İSTANBUL,PENDİK,YENI,79.00,0,5,10,100,1,1,...,375.11,448.02,464.64,11.43,379.46,504.23,398.69,662.86,109.80,5134.80
1,İSTANBUL,PENDİK,FATIH,83.00,2,16,5,100,0,1,...,375.11,448.02,464.64,11.43,379.46,504.23,398.69,662.86,109.80,5134.80
2,İSTANBUL,PENDİK,ESENYALI,82.00,0,5,3,100,0,1,...,375.11,448.02,464.64,11.43,379.46,504.23,398.69,662.86,109.80,5134.80


In [37]:
df_coordinates_nominatim

,ILI,ILCESI,MAHALLESI,LATITUDE,LONGITUDE
0,İSTANBUL,PENDİK,KAVAKPINAR,40.88,29.29
1,İSTANBUL,PENDİK,KURTKÖY,40.91,29.30
2,İSTANBUL,PENDİK,VELIBABA,40.90,29.25
3,İSTANBUL,PENDİK,KAYNARCA,40.88,29.26
4,İSTANBUL,PENDİK,BAHÇELIEVLER,40.88,29.24
5,İSTANBUL,PENDİK,FEVZI ÇAKMAK,40.88,29.26
6,İSTANBUL,PENDİK,GÜZELYALI,40.86,29.28
7,İSTANBUL,PENDİK,YENIŞEHIR,40.93,29.30
8,İSTANBUL,PENDİK,ÇINARDERE,40.89,29.24
9,İSTANBUL,PENDİK,GÜLLÜ BAĞLAR,40.90,29.27


In [38]:
sub_df_combined = df_sub.merge(features_merged, on="MAHALLESI")
sub_df_combined.to_csv("sub_df_combined.csv", index=False)

In [39]:
train_copy = train_df.copy()

In [40]:
train_copy = train_copy.replace(encode_transform)

In [41]:
sub_df_combined = missing_mahalle_filling_geo(sub_df_combined, df_coordinates_nominatim)

   KAYIP_MAHALLE AYNI_ILCE_MAHALLE  DISTANCE
4           YENI      BAHÇELIEVLER      0.38
15          YENI              BATI      0.67
11          YENI              DOĞU      0.74
21          YENI     SAPAN BAĞLARI      0.82
8           YENI         ÇINARDERE      1.46
20          YENI        DUMLUPINAR      1.77
3           YENI          KAYNARCA      2.23
22          YENI          DOLAYOBA      2.42
5           YENI      FEVZI ÇAKMAK      2.83
2           YENI          VELIBABA      3.08
18          YENI           ESENLER      3.52
12          YENI          ÇAMÇEŞME      3.55
10          YENI           YAYALAR      3.75
9           YENI      GÜLLÜ BAĞLAR      3.96
0           YENI        KAVAKPINAR      4.71
6           YENI         GÜZELYALI      4.99
17          YENI         SÜLÜNTEPE      5.16
19          YENI         ORHANGAZI      5.28
14          YENI            ÇAMLIK      6.70
13          YENI      AHMET YESEVI      6.75
1           YENI           KURTKÖY      6.77
7         

   KAYIP_MAHALLE AYNI_ILCE_MAHALLE  DISTANCE
19         FATIH         ORHANGAZI      0.72
13         FATIH      AHMET YESEVI      0.94
6          FATIH         GÜZELYALI      1.53
0          FATIH        KAVAKPINAR      1.75
12         FATIH          ÇAMÇEŞME      2.34
18         FATIH           ESENLER      2.87
5          FATIH      FEVZI ÇAKMAK      3.07
3          FATIH          KAYNARCA      3.66
9          FATIH      GÜLLÜ BAĞLAR      3.80
22         FATIH          DOLAYOBA      4.02
10         FATIH           YAYALAR      4.67
20         FATIH        DUMLUPINAR      4.96
1          FATIH           KURTKÖY      5.18
11         FATIH              DOĞU      5.26
17         FATIH         SÜLÜNTEPE      5.31
2          FATIH          VELIBABA      5.48
4          FATIH      BAHÇELIEVLER      5.54
8          FATIH         ÇINARDERE      5.73
15         FATIH              BATI      5.76
21         FATIH     SAPAN BAĞLARI      6.70
14         FATIH            ÇAMLIK      6.86
7         

The function we created before is giving the closest neighbourhoods value to the missing neighbourhoods.

In [42]:
sub_df_combined = sub_df_combined.replace(encode_transform)

In [43]:
sub_df_combined.head(3)

,ILI,ILCESI,MAHALLESI,DAIRE_YASAL_M2,KAT_NO,BINA_YASI,KAT_ADEDI,İNŞAAT_SEVİYESİ,ASANSÖR,DAIRE_SALON,...,playground,police,school,stadium,station,forest,hospital,park,parking,residential
0,0,0,1,79.00,0,5,10,100,1,1,...,0.00,0.00,2.00,0.00,2.00,0.00,0.00,1236.23,4480.33,3264614.23
1,0,0,1,113.00,4,11,9,100,1,1,...,0.00,0.00,2.00,0.00,2.00,0.00,0.00,1236.23,4480.33,3264614.23
2,0,0,1,91.00,2,5,7,100,1,1,...,0.00,0.00,2.00,0.00,2.00,0.00,0.00,1236.23,4480.33,3264614.23


In [44]:
train_copy.MAHALLESI.unique()

array([11, 13, 17, 12,  1,  7,  9, 19, 22,  8, 18,  4, 21,  0, 20,  2, 10,
       16,  6, 14,  5, 15], dtype=int64)

In [45]:
sub_df_combined.to_csv("sub_combined_geolocated.csv", index=False)

We are creating the model for our predictions.

In [46]:
X = train_copy.drop(["NIHAI_DEGERI"],axis=1)
y = train_copy["NIHAI_DEGERI"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2151)

model_ngb = NGBRegressor(natural_gradient=True,
                        learning_rate = 0.01,
                        Dist=Normal,
                        Score=MLE,
                        verbose_eval=100,
                        random_state=2151)

model_ngb.fit(X_train, y_train)

y_train_ngb = model_ngb.pred_dist(X_train)
y_test_ngb = model_ngb.pred_dist(X_test)

print("Train SMAPE: ", smape_score(y_train, y_train_ngb.loc))
print("Test SMAPE: ", smape_score(y_test, y_test_ngb.loc))

[iter 0] loss=17.1626 val_loss=0.0000 scale=1.0000 norm=3892881.3017
[iter 100] loss=16.1995 val_loss=0.0000 scale=2.0000 norm=3569483.4604
[iter 200] loss=15.6839 val_loss=0.0000 scale=1.0000 norm=1271147.8520
[iter 300] loss=15.4799 val_loss=0.0000 scale=1.0000 norm=1139294.8397
[iter 400] loss=15.3669 val_loss=0.0000 scale=1.0000 norm=1063674.9789
Train SMAPE:  20.997596800393964
Test SMAPE:  23.526210393906997


In [48]:
sub_pred = model_ngb.pred_dist(sub_df_combined)

sub_df_pred = pd.DataFrame({"NIHAI_DEGERI": sub_pred.loc.astype(int)})

In [49]:
sub_df_pred

,NIHAI_DEGERI
0,2850152
1,3349691
2,3578394
3,2854441
4,3210615
...,...
131,8217540
132,10909102
133,10995168
134,12473352
